In [21]:
import pandas as pd
import pymysql

In [2]:
# Sql 서버의 종류 입력
_db =pymysql.connect(
    user ="root",
    password='비밀번호',
    host='localhost',
    db='db명',
    port=3306

)

In [3]:
# cursor 변수 생성
# sql 퀴리문을 실행 시키기 위한 필수 변수
# Dictcursor는 sql 쿼리문이 실행되고 나온 데이터를 딕셔너리 형태로 받는다.

cursor= _db.cursor(pymysql.cursors.DictCursor)

## sql 함수화
1. 첫 번째 함수
    - 매개변수 1개(sql 쿼리문)
    - sql 쿼리문을 실행하고 결과를 데이터프레임으로 변경하여 return
2. 두 번째 함수
    - 매개변수 1개(sql 쿼리문)
    - sql 쿼리문을 실행하고 데이터베이스를 변경
3. 세 번째 함수
    - 1번 함수와 2번 함수를 하나의 함수에서 실행이 될 수 있다고록 생성
    - select일 경우에만 데이터프레임으로 반환
    - select가 아닌 DML문법(insert,update,delete)경우에는 "Qurrey Ok"로 반환됨
    - DCL문법은 자동으로 커밋됨

In [4]:
def _first(sql):
    # 쿼리문 실행
    cursor.execute(sql)
    # 쿼리문에 대한 결과
    result=cursor.fetchall()
    return pd.DataFrame(result)

In [5]:
def _second(sql,values):
    cursor.execute(sql,values)
    _db.commit() 

In [7]:
_sql=("""
    insert into
    user_info
    values
    (%s,%s,%s,%s,%s)
""")
values=['test12','12234','test10','01077777778',1]

_second(_sql,values)

In [11]:
def sql_query(sql, values=[]):
    try:
        # 데이터베이스 연결 및 커서 생성
        _db =pymysql.connect(
            user ="root",
            password='1234',
            host='localhost',
            db='ubion7',
            port=3306) 
        # 여기에 데이터베이스 연결 코드 작성, 함수에서 이부분만 수정하면 된다
        cursor = _db.cursor()

        if (sql.replace('\n','').strip().startswith('select')):
            cursor.execute(sql, values)
            result = cursor.fetchall()
            result = pd.DataFrame(result)
        else:
            cursor.execute(sql, values)
            _db.commit()
            result = "Query Ok"

        # 커서와 데이터베이스 연결 닫기
        cursor.close()
        _db.close()

        return result

    except Exception as e:
        return str(e)


- 3번째 함수에서 select 문장을 실행할때

In [10]:
sql='''
    select
    *
    from
    user_info
'''
sql_query(sql)

,0,1,2,3,4
0,test,1234,test,01012345678,1
1,test11,12234,test10,01077777777,1
2,test12,12234,test10,01077777778,1
3,test2,1234,test2,01011111111,2
4,test3,1234,test3,01099999999,1
5,test4,1111,test4,01011111111,1
6,test5,1234,test5,01011118888,1
7,test6,1274,test6,01011119999,1
8,test7,1274,test6,01011119999,1


- 3번째 함수에서 select이외의 DML문장을 실행할때

In [12]:
_sql=("""
    delete
    from
    user_info
    where
    user_id =%s
    
""")
# where in (%s,%s,%s)
values=['test11']

sql_query(_sql,values)

'Query Ok'

- 삭제 후 다시 select 문장을 실행

In [13]:
sql='''
    select
    *
    from
    user_info
'''
sql_query(sql)

,0,1,2,3,4
0,test,1234,test,01012345678,1
1,test12,12234,test10,01077777778,1
2,test2,1234,test2,01011111111,2
3,test3,1234,test3,01099999999,1
4,test4,1111,test4,01011111111,1
5,test5,1234,test5,01011118888,1
6,test6,1274,test6,01011119999,1
7,test7,1274,test6,01011119999,1


In [42]:
import mod_sql
import imp
imp.reload(mod_sql)

C:\Users\dgh06\AppData\Local\Temp\ipykernel_14864\2748340258.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


<module 'mod_sql' from 'c:\\Users\\dgh06\\OneDrive\\문서\\GitHub\\SQL\\mod_sql\\__init__.py'>

In [44]:
test=mod_sql.Database(_host='localhost',_user='root',_pass='1234',_db='ubion7',_port=3306)

In [45]:
sql='''
    select
    `Name` as '이름',
    `Phone` as '휴대폰번호'
    from
    example
    where
    `Gender` = %s
    limit 3
'''
values=['male']
test.sql_query(sql,values)

# limit 3 : 3개만 뽑겠다
# 콜럼의 이름에 ``붙이는게 좋다-->띄어쓰기 방지!-->앞에 하나의 단어만 인식함

,이름,휴대폰번호
0,A,010-1234-5678
1,C,010-1234-5680
2,E,010-1234-5682


In [46]:
sql='''
    select
    createDt as '등록일시',
    deathCnt as '사망자',
    decideCnt as '감염자'
    from
    corona

'''
test.sql_query(sql)

,등록일시,사망자,감염자
0,2022-06-08 09:09:05.982,24305,18188200
1,2022-06-07 09:09:00.897,24299,18174842
2,2022-06-06 09:00:06.734,24279,18168670
3,2022-06-05 08:53:19.426,24258,18163648
4,2022-06-04 08:56:49.219,24238,18153814
...,...,...,...
815,2020-03-14 00:00:00.000,72,8086
816,2020-03-13 00:00:00.000,67,7979
817,2020-03-12 00:00:00.000,66,7869
818,2020-03-11 00:00:00.000,60,7755


In [48]:
sql="""
    select
    `Country` as '국가',
    `Units Sold` as '판매개수',
    `Unit Price` as `가격`,
    `Unit Cost` as `원가`,
    (`Units Sold` * `Unit Price`) as `총수익`,
    (`Units Sold` * `Unit Cost`) as '총비용',
    ((`Units Sold` * `Unit Price`) - (`Units Sold` * `Unit Cost`)) as `총이윤`
    from 
    `Sales Records`

"""
test.sql_query(sql)

,국가,판매개수,가격,원가,총수익,총비용,총이윤
0,Chad,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Latvia,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Pakistan,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Democratic Republic of the Congo,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Czech Republic,3491,47.45,31.79,165647.95,110978.89,54669.06
...,...,...,...,...,...,...,...
9995,Laos,8597,47.45,31.79,407927.65,273298.63,134629.02
9996,Liechtenstein,562,437.20,263.33,245706.40,147991.46,97714.94
9997,Democratic Republic of the Congo,2524,154.06,90.93,388847.44,229507.32,159340.12
9998,South Africa,8706,421.89,364.69,3672974.34,3174991.14,497983.20
